# 핵심 기능

## TypeDict
1. dict와 TypedDict의 주요 차이점
    - 타입 검사 : 정적 타입 검사를 제공, 코드 작성 시 IDE나 타입 체커가 오류를 미리 잡아낼 수 있음
    - 키와 값의 타입 :  키에 대해 구체적인 타입을 지정할 수 있음
    - 유연성 : 정의된 구조를 따라야 합니다. 추가적인 키는 타입 오류를 발생

In [1]:
from typing import Dict, TypedDict

sample_dict: Dict[str, str] = {
    "name": "테디",
    "age": "30",
    "job": "개발자",
}


# TypedDict 사용
class Person(TypedDict):
    name: str
    age: int 
    job: str


typed_dict: Person = {"name": "셜리", "age": 25, "job": "디자이너"}

In [2]:
# dict의 경우
sample_dict["age"] = 35  # 문자열에서 정수로 변경되어도 오류 없음
sample_dict["new_field"] = "추가 정보"  # 새로운 필드 추가 가능

# TypedDict의 경우
typed_dict["age"] = 35  # 정수형으로 올바르게 사용
typed_dict["age"] = "35"  # 타입 체커가 오류를 감지함
typed_dict["new_field"] = (
    "추가 정보"
)

## Annotated
이 문법은 타입 힌트에 메타데이터를 추가할 수 있게 해줍니다. Annotated를 사용하는 주요 이유
- ex)
    - `name: Annotated[str, "이름"]`
    - `age: Annotated[int, "나이"]`
```python
from typing import Annotated

variable: Annotated[Type, metadata1, metadata2, ...]
```
- `Type`: 기본 타입 (예: int, str, List[str] 등)
- `metadata1, metadata2, ...`: 추가하고자 하는 메타데이터

In [4]:
# 예시

from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이 (0-150)"]

In [6]:
# Pydantic과 함께 사용

from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError


class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=18, lt=65, description="나이 (19-64세)")]
    salary: Annotated[
        float, Field(gt=0, lt=10000, description="연봉 (단위: 만원, 최대 10억)")
    ]
    skills: Annotated[
        List[str], Field(min_items=1, max_items=10, description="보유 기술 (1-10개)")
    ]


# 유효한 데이터로 인스턴스 생성
try:
    valid_employee = Employee(
        id=1, name="테디노트", age=30, salary=1000, skills=["Python", "LangChain"]
    )
    print("유효한 직원 데이터:", valid_employee)
except ValidationError as e:
    print("유효성 검사 오류:", e)

# 유효하지 않은 데이터로 인스턴스 생성 시도
try:
    invalid_employee = Employee(
        name="테디",  # 이름이 너무 짧음
        age=17,  # 나이가 범위를 벗어남
        salary=20000,  # 급여가 범위를 벗어남
        skills="Python",  # 리스트가 아님
    )
except ValidationError as e:
    print("유효성 검사 오류:")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")

유효한 직원 데이터: id=1 name='테디노트' age=30 salary=1000.0 skills=['Python', 'LangChain']
유효성 검사 오류:
- id: Field required
- name: String should have at least 3 characters
- age: Input should be greater than 18
- salary: Input should be less than 10000
- skills: Input should be a valid list


## LangGraph에서의 사용(add_messages)
- add_messages 는 LangGraph 에서 메시지를 리스트에 추가하는 함수입니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages

class MyData(TypedDict):
    messages: Annotated[list, add_messages]

In [7]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages


class MyData(TypedDict):
    messages: Annotated[list, add_messages]

### add_messages

- messages 키는 add_messages 리듀서 함수로 주석이 달려 있으며, 이는 LangGraph에게 기존 목록에 새 메시지를 추가하도록 지시합니다.

- 주석이 없는 상태 키는 각 업데이트에 의해 덮어쓰여져 가장 최근의 값이 저장됩니다.

- add_messages 함수는 2개의 인자(left, right)를 받으며 좌, 우 메시지를 병합하는 방식으로 동작합니다.

In [10]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

result1 = add_messages(msgs1, msgs2)
print(result1)

# 동일한 ID를 가진 메시지 대체 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [HumanMessage(content="반갑습니다~", id="1")]

result2 = add_messages(msgs1, msgs2)
print(result2)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]
[HumanMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='1')]
